In [1]:
import graphviz
import ipywidgets as widgets
from IPython.display import display
import random
from datetime import datetime

In [2]:
class IDevice():
    def state_regen(self): pass
    def broken(self) -> bool: pass

class Device(IDevice):
    def __init__(self, breakdown_probability):
        self.breakdown_probability = breakdown_probability
        self.type = 'Device'
    def state_regen(self):
        random.seed(datetime.now().timestamp())
        self.number = random.random()
        self.isBroken = self.number < self.breakdown_probability
    def broken(self):
        return self.isBroken

class Parallel(IDevice):
    def __init__(self, config):
        self.parallel = config
        self.type = "Parallel"
    def state_regen(self):
        for device in self.parallel:
            device.state_regen()
    def broken(self) -> bool:
        for device in self.parallel:
            if not device.broken():
                return False
        return True

class Pipeline(IDevice):
    def __init__(self, config):
        self.pipeline = config
        self.type = "Pipeline"
    def state_regen(self):
        for device in self.pipeline:
            device.state_regen()
    def broken(self):
        for device in self.pipeline:
            if device.broken():
                return True
        return False
    def Render(self, device = None, indent=0, prop="breakdown_probability"):
        if device is None:
            device = self
        tab = indent*"\t"
        match(device.type):
            case "Device":
                print(f"{tab}Device {getattr(device, prop)},")
            case "Pipeline":
                print(f"{tab}"+"{")
                for item in device.pipeline:
                    self.Render(item, indent + 1, prop)
                print(f"{tab}" + "},")
            case "Parallel":
                print(f"{tab}[")
                for item in device.parallel:
                    self.Render(item, indent + 1, prop)
                print(f"{tab}],")
            case _:
                print(device.type)

In [7]:
pipeline = Pipeline([
    Device(0.22),
    Parallel([
        Pipeline([
            Device(0.06),
            Parallel([
                Device(0.11),
                Device(0.11)
            ])
        ]),
        Device(0.18)
    ]),
    Device(0.21)
])

In [5]:
pipeline.Render(pipeline, 0)

{
	Device 0.22,
	[
		{
			Device 0.06,
			[
				Device 0.11,
				Device 0.11,
			],
		},
		Device 0.18,
	],
	Device 0.21,
},


In [13]:
pipeline.state_regen()
pipeline.broken()

True

In [10]:
pipeline.Render(pipeline, 0, "isBroken")

{
	Device False,
	[
		{
			Device False,
			[
				Device False,
				Device False,
			],
		},
		Device False,
	],
	Device False,
},


In [319]:
pipeline.Render(pipeline, 0, "number")

{
	Device 0.7003977277317958,
	[
		{
			Device 0.7003977277317958,
			[
				Device 0.7003977277317958,
				Device 0.7003977277317958,
			],
		},
		Device 0.7003977277317958,
	],
	Device 0.7003977277317958,
},


In [14]:
n = 10000
k = 0
for i in range(n):
    pipeline.state_regen()
    k += pipeline.broken()
print(f"broken {k} times")
print(f"with {k/n*100}% probability to break")
print(f"and with {(1-k/n)*100}% probability to live")

broken 2445 times
with 24.45% probability to break
and with 75.55000000000001% probability to live
